# A word about stimulus contrast

For users who want to pursue this line of research, keep in mind that the stimulus resolution (ppd) and the amount of padding affect our stimulus definition when defined in rms contrast.
There is a linear relationship between rms contrast and Michelson contrast within one edge type, i.e. an increase in rms contrast linearly translate to mc contrast.

Since we keep stimulus resolution and padding constant in our experiment and simulations, this is not relevant.
However, this might be relevant for further simulations in which users want to / need to change the stimulus definition to match stimulus definitions and model requirements.


## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from stimupy.noises.whites import white as create_whitenoise
from stimupy.noises.narrowbands import narrowband as create_narrownoise
from stimupy.noises.naturals import one_over_f as create_pinknoise

sys.path.insert(1, '../mechanistic_models')
from functions import pull_noise_mask, create_edge, add_padding
from experimental_data.exp_params import stim_params as sparams

## Helper functions

In [ ]:
def rms(img):
    return img.std() / img.mean()

def mc(img):
    return (img.max() - img.min()) / (img.max() + img.min())

# Stimulus parameters

In [ ]:
stimSize = sparams["stim_size"]
ppd = sparams["ppd"]
meanLum = sparams["mean_lum"]

# Edge parameters
edgeConds = sparams["edge_widths"]
edgeExponent = sparams["edge_exponent"]

# Noise parameters
noiseConds = sparams["noise_types"]
noiseContrast = sparams["noise_contrast"]

# Read psychophysical data
df = pd.read_csv("../experimental_data/expdata_pooled.txt", sep=" ")
df

# Changing ppd

In [ ]:
def changeEdgePPD(ppd, sparams):
    c = df[(df.noise==noiseConds[2]) & (df.edge==edgeConds[0])].contrasts.iloc[3]
    e = create_edge(c, edgeConds[0], {**sparams, "ppd": ppd})
    print("PPD=%.0f\t rms: %.3f; mc: %.3f; min: %.1f; max: %.1f" % (ppd, rms(e), mc(e), e.min(), e.max()))
    return e

plt.figure(figsize=(10,4))
plt.subplot(121); plt.imshow(changeEdgePPD(44,  sparams), cmap='gray'), plt.colorbar()
plt.subplot(122); plt.imshow(changeEdgePPD(500, sparams), cmap='gray'), plt.colorbar()
plt.show()

# Padding

In [ ]:
def changeEdgePad(pad, sparams):
    c = df[(df.noise==noiseConds[2]) & (df.edge==edgeConds[0])].contrasts.iloc[3]
    e = create_edge(c, edgeConds[0], sparams)
    e = add_padding(e, pad, e.mean())
    print("Pad=%.0f\t rms: %.3f; mc: %.3f; min: %.1f; max: %.1f" % (pad, rms(e), mc(e), e.min(), e.max()))
    return e

plt.figure(figsize=(10,4))
plt.subplot(121); plt.imshow(changeEdgePad(0,  sparams), cmap='gray'), plt.colorbar()
plt.subplot(122); plt.imshow(changeEdgePad(50, sparams), cmap='gray'), plt.colorbar()
plt.show()

# Relationship Michelson + RMS-contrast for all stimuli

In [ ]:
vrms = np.zeros([len(noiseConds), len(edgeConds), 5])
vmc = np.zeros([len(noiseConds), len(edgeConds), 5])

for ni, n in enumerate(noiseConds):
    for ei, e in enumerate(edgeConds):
        for ci, c in enumerate(df[(df.noise==n) & (df.edge==e)].contrasts):
            es = create_edge(c, e, sparams)
            vrms[ni, ei, ci] = rms(es)
            vmc[ni, ei, ci] = mc(es)

In [ ]:
for ei in range(len(edgeConds)):
    plt.plot(vrms[:,ei,:].flatten(), vmc[:,ei,:].flatten(), '.')
    plt.xlabel("rms"); plt.ylabel("mc"); plt.legend(edgeConds)